# **Main ICM Final Project Jupyter Notebook**

## **Needed Theory**
 - Convolutions
 - FFT

Derive those formulae in the context of this project and cite then OK

## **Justification for Choice of Images**

 - `antimeme.png`: A vector-like, clean image with large flat color regions and strong contrast. This makes it easier to visually assess blur and to recover with regularization.

- `save_me.png`: A real-life photo with natural lighting, surface texture, sensor noise, and overlaid Chinese text plus artificial purple scribble. These factors introduce mixed frequencies and uneven illumination, making deconvolution and denoising more difficult.

- `this_is_fine.png`: A comic drawing that is not vectorized and contains bold ink lines, but also visible grain and scan noise. It lies between the other two in difficulty: more complex than `antimeme.png` but cleaner than the real photo `save_me.png`.

We hypothesize the difficulty of restoring these images after encryption to be in this order: `save_me.png` > `this_is_fine.png` > `antimeme.png`

## **Justification for Choice of Private Key Image**

`private_key.png` is a real-life photo, which means the natural lighting may create enough noise. As the photo subject is mainly green, this will ensure the kernels for R, G, and B will be different. All in all, these factors should make it a less predictable kernel.

## <u>**Experiment Plan**</u>

## **Experiments**

Repeat the following kernels for all three images. We consider R, G, B as three separate channels needed for kernels and do not do grayscale, for better visual differentiation.

### **Baseline: "Transformations" (No Private Key Convolutions)**
 - Gaussian Blur
 
 - Motion Blur

 - Box Blur

### **New: "Encryptions" (Private Key Convolutions)**
 - Random Kernel from a seed (seed = private key), of course fix the seed across all three images
 
 - Sequence of Kernels
 
 - Private Key Image-derived Kernel via `private_key.png`

## **Data Extracted**

For each image + kernel + parameter setting, store:

### Quantitative Data
 - **Image ID**: `antimeme`, `this_is_fine`, `save_me`
 
 - **Kernel type**: `Gaussian`, `Motion`, `Box`, `RandomSeed`, `KeyImage`
 
 - **Kernel parameters**: size, $\sigma$ (for Gaussian), length/angle (for motion), seed, which key image
 - **Noise level** $\sigma$ (for added Gaussian noise)
 
 - **Regularization parameter** $\lambda$

 - **MSE** between original and recovered (over all RGB channels)
 
 - **PSNR** derived from MSE

### Qualitative Data
 - **The Images**: original vs encrypted vs noisy vs recovered
 
 - Notes on **visual artifacts** such as ringing around edges, halo artifacts, oversmoothing, color shifts, "ghosting" shapes, complete failure (looks random)

## **Methodology**

### Summary
This project studies how image reconstruction behaves under different convolution-based transformations, including both standard (public) kernels and “encrypted” private-key kernels. We evaluate reconstruction accuracy when the correct kernel is known, and investigate the sensitivity of the inverse problem by testing recovery with intentionally incorrect kernels. We do not perform blind kernel estimation; all recovery assumes access to either the true kernel or a deliberately mismatched one.

### 1. Preprocessing
- Load each image (`antimeme`, `this_is_fine`, `save_me`) as an RGB array of size $512 \times 512 \times 3$.
- Normalize pixel values to $[0,1]$.
- No grayscale conversion is performed; convolution is applied independently to each R, G, B channel.

### 2. Apply Operator (Forward Model)
For each image and each kernel type:

#### Baseline Transformations (public kernels)
These kernels are fully known:
- Gaussian blur (fixed size and $\sigma$)
- Motion blur (fixed length and angle)
- Box blur

#### Encryption Operators (private-key kernels)
These kernels are not publicly known:
- Random kernel generated from a fixed seed (seed = private key)
- Key-image-derived kernel from `private_key.png`

Each kernel is applied channel-wise to form:
$$
y = Kx
$$

Then additive Gaussian noise is applied:
$$
z = y + \text{noise}
$$

Noise is added to make the inverse problem realistic.

### 3. Recovery (Inverse Problem)

#### Correct-Kernel Recovery
For each baseline or private-key kernel, perform non-blind deconvolution by solving:

$$
\hat{x} = \arg\min_x \|Kx - z\|^2 + \lambda \|Lx\|^2
$$

where:
- $K$ = the known convolution operator
- $L$ = a roughening operator (finite differences)
- $\lambda$ = regularization strength

The solution is computed channel-wise using:
- an FFT-based approximate inverse.

#### Incorrect-Kernel Recovery (for encryption tests)
To demonstrate key-dependence, we intentionally recover using a wrong kernel:
- wrong random seed
- wrong kernel size
- Gaussian or motion blur instead of the private kernel

We solve the same inverse problem using $\tilde{K}$:

$$
\hat{x}_{wrong} = \arg\min_x \|\tilde{K}x - z\|^2 + \lambda \|Lx\|^2
$$

This typically produces severe reconstruction failure and demonstrates that successful recovery requires the correct key.

We do not attempt to guess or estimate kernels.

### 4. Evaluation

#### Quantitative Metrics
- MSE between original and recovered RGB image:
$$
\text{MSE} = \frac{1}{N} \sum_{i=1}^N (x_i - \hat{x}_i)^2
$$

- PSNR derived from MSE:
$$
\text{PSNR} = 10 \log_{10} \left( \frac{1}{\text{MSE}} \right)
$$

(assuming images are normalized to $[0,1]$).

#### Recorded Metadata
For each experiment, record:
- kernel type
- kernel parameters
- noise level $\sigma$
- regularization $\lambda$
- whether the kernel used for recovery was correct or wrong

#### Qualitative Comparison
For each experiment, visually compare:
- original image
- transformed/encrypted image
- noisy observation
- recovered image (correct key)
- recovered image (wrong key)

and note artifacts such as ringing, oversmoothing, color distortion, or complete failure.